In [2]:
import sql
import pandas as pd
from functools import reduce

In [16]:
q='''select * from alert a
join project p on a.project_id = p.id
join memory_error me on a.alert_type_id = me.alert_type_id
where is_invalid=0
and me.memory =1
order by name;'''
root=pd.DataFrame(sql.execute(q)) 
#root.head()
total=root.groupby('name')[['cid']].count()
total.rename(columns={'cid':'total'},inplace=True)
intentional=root[root['classification']=='Intentional'].groupby('name')[['cid']].count()
intentional.rename(columns={'cid':'i'},inplace=True)
fp=root[root['classification']=='False Positive'].groupby('name')[['cid']].count()
fp.rename(columns={'cid':'fp'},inplace=True)
alive = root[root['status']=='New'].groupby('name')[['cid']].count()
alive.rename(columns={'cid':'alive'},inplace=True)
df=reduce(lambda x,y:pd.merge(x,y,on='name'),[total,intentional,fp,alive])
q='''select name, count(*) as c from alert a
    join project p on a.project_id = p.id
    join memory_error me on a.alert_type_id = me.alert_type_id
    join actionability a2 on a.id = a2.alert_id
    where is_invalid = 0
    and status='Fixed'
    and a2.actionability=0
    and memory =1
    and a2.file_deleted is null
    and a2.file_renamed is null
    and a2.suppression is null
    group by name
    order by name;
    '''
unact=pd.DataFrame(sql.execute(q))
df=reduce(lambda x,y:pd.merge(x,y,on='name'),[total,intentional,fp,alive,unact])
df.i=round((df.i/df.total)*100,2)
df.fp=round((df.fp/df.total)*100,2)
df.alive=round((df.alive/df.total)*100,2)
df.c=round((df.c/df.total)*100,2)
df.i=df.i.astype(str)+"%"
df.fp=df.fp.astype(str)+"%"
df.alive=df.alive.astype(str)+"%"
df.c=df.c.astype(str)+"%"
df=df.rename(columns={'total':'total memory','i':'intentional memory','fp': 'false positive memory','c':'unknown elimination memory','alive':'alive memory'})
memory=df
memory

,name,total memory,intentional memory,false positive memory,alive memory,unknown elimination memory
0,Firefox,5407,3.85%,9.78%,8.73%,25.02%
1,Kodi,640,9.22%,10.0%,11.56%,9.69%
2,Linux,7600,1.67%,4.17%,20.63%,29.09%
3,Samba,1734,0.81%,2.83%,17.65%,37.08%
4,ovirt-engine,1273,5.58%,49.41%,8.96%,8.56%


In [18]:
q='''select * from alert a
join project p on a.project_id = p.id
join memory_error me on a.alert_type_id = me.alert_type_id
where is_invalid=0
and me.memory is null
order by name;'''
root=pd.DataFrame(sql.execute(q)) 
#root.head()
total=root.groupby('name')[['cid']].count()
total.rename(columns={'cid':'total'},inplace=True)
intentional=root[root['classification']=='Intentional'].groupby('name')[['cid']].count()
intentional.rename(columns={'cid':'i'},inplace=True)
fp=root[root['classification']=='False Positive'].groupby('name')[['cid']].count()
fp.rename(columns={'cid':'fp'},inplace=True)
alive = root[root['status']=='New'].groupby('name')[['cid']].count()
alive.rename(columns={'cid':'alive'},inplace=True)
df=reduce(lambda x,y:pd.merge(x,y,on='name'),[total,intentional,fp,alive])
q='''select name, count(*) as c from alert a
    join project p on a.project_id = p.id
    join memory_error me on a.alert_type_id = me.alert_type_id
    join actionability a2 on a.id = a2.alert_id
    where is_invalid = 0
    and status='Fixed'
    and a2.actionability=0
    and memory is null
    and a2.file_deleted is null
    and a2.file_renamed is null
    and a2.suppression is null
    group by name
    order by name;
    '''
unact=pd.DataFrame(sql.execute(q))
df=reduce(lambda x,y:pd.merge(x,y,on='name'),[total,intentional,fp,alive,unact])
df.i=round((df.i/df.total)*100,2)
df.fp=round((df.fp/df.total)*100,2)
df.alive=round((df.alive/df.total)*100,2)
df.c=round((df.c/df.total)*100,2)
df.i=df.i.astype(str)+"%"
df.fp=df.fp.astype(str)+"%"
df.alive=df.alive.astype(str)+"%"
df.c=df.c.astype(str)+"%"
df=df.rename(columns={'total':'total nonmemory','i':'intentional nonmemory','fp': 'false positive nonmemory','c':'unknown elimination nonmemory','alive':'alive nonmemory'})
nonmemory=df
nonmemory

,name,total nonmemory,intentional nonmemory,false positive nonmemory,alive nonmemory,unknown elimination nonmemory
0,Firefox,7538,10.27%,5.76%,12.55%,22.35%
1,Kodi,1753,15.46%,1.14%,16.26%,13.69%
2,Linux,11919,5.54%,3.72%,25.14%,22.34%
3,Samba,2773,2.24%,2.27%,22.14%,46.34%
4,ovirt-engine,1716,9.27%,38.46%,4.25%,13.93%


In [9]:
q='''select name, count(*) as memory from alert a
    join project p on a.project_id = p.id
    join memory_error me on a.alert_type_id = me.alert_type_id
    where is_invalid = 0
    and classification ='Intentional'
    and memory = 1
    group by name
    order by name; '''
tma=pd.DataFrame(sql.execute(q))
tma

,name,memory
0,Firefox,153
1,Kodi,42
2,Linux,64
3,ovirt-engine,67
4,Samba,14


In [4]:
q='''select name, count(*) as non_memory from alert a
    join project p on a.project_id = p.id
    join memory_error me on a.alert_type_id = me.alert_type_id
    where is_invalid = 0
    and classification ='Intentional'
    and memory is null
    group by name
    order by name; '''
tnma=pd.DataFrame(sql.execute(q))
tnma

,name,non_memory
0,Firefox,829
1,Kodi,288
2,Linux,723
3,ovirt-engine,163
4,Samba,62


In [12]:
df = reduce(lambda x,y: pd.merge(x,y, on='name', how='outer'),[ta,tma,tnma])
df.memory= round((df.memory/df.total) * 100,2)
df.non_memory= round((df.non_memory/df.total) * 100,2)
df['memory']=df['memory'].astype(str)+"%"
df['non_memory']=df['non_memory'].astype(str)+"%"
df=df.rename(columns={'total':'intentional'})
df

,name,intentional,memory,non_memory
0,Firefox,982,15.58%,84.42%
1,Kodi,330,12.73%,87.27%
2,Linux,787,8.13%,91.87%
3,ovirt-engine,230,29.13%,70.87%
4,Samba,76,18.42%,81.58%


In [19]:
q='''select name, count(*) as total from alert a
    join project p on a.project_id = p.id
    join memory_error me on a.alert_type_id = me.alert_type_id
    where is_invalid = 0
    and classification ='False Positive'
    group by name
    order by name; '''
ta=pd.DataFrame(sql.execute(q))
q='''select name, count(*) as memory from alert a
    join project p on a.project_id = p.id
    join memory_error me on a.alert_type_id = me.alert_type_id
    where is_invalid = 0
    and classification ='False Positive'
    and memory = 1
    group by name
    order by name; '''
tma=pd.DataFrame(sql.execute(q))
q='''select name, count(*) as non_memory from alert a
    join project p on a.project_id = p.id
    join memory_error me on a.alert_type_id = me.alert_type_id
    where is_invalid = 0
    and classification ='False Positive'
    and memory is null
    group by name
    order by name; '''
tnma=pd.DataFrame(sql.execute(q))
df = reduce(lambda x,y: pd.merge(x,y, on='name', how='outer'),[ta,tma,tnma])
df.memory= round((df.memory/df.total) * 100,2)
df.non_memory= round((df.non_memory/df.total) * 100,2)
df['memory']=df['memory'].astype(str)+"%"
df['non_memory']=df['non_memory'].astype(str)+"%"
df=df.rename(columns={'total':'false positive'})
df

,name,false positive,memory,non_memory
0,Firefox,963,51.3%,48.7%
1,Kodi,84,75.0%,25.0%
2,Linux,760,33.55%,66.45%
3,ovirt-engine,1289,48.56%,51.44%
4,Samba,112,43.75%,56.25%


In [18]:
q='''select name, count(*) as total from alert a
    join project p on a.project_id = p.id
    join memory_error me on a.alert_type_id = me.alert_type_id
    join actionability a2 on a.id = a2.alert_id
    where is_invalid = 0
    and status='Fixed'
    and a2.actionability=0
    and a2.file_deleted is null
    and a2.file_renamed is null
    and a2.suppression is null
    group by name
    order by name;
    '''
ta=pd.DataFrame(sql.execute(q))
q='''select name, count(*) as memory from alert a
    join project p on a.project_id = p.id
    join memory_error me on a.alert_type_id = me.alert_type_id
    join actionability a2 on a.id = a2.alert_id
    where is_invalid = 0
    and status='Fixed'
    and a2.actionability=0
    and a2.file_deleted is null
    and a2.file_renamed is null
    and a2.suppression is null
    and me.memory=1
    group by name
    order by name;
    '''
tma=pd.DataFrame(sql.execute(q))
q='''select name, count(*) as non_memory from alert a
    join project p on a.project_id = p.id
    join memory_error me on a.alert_type_id = me.alert_type_id
    join actionability a2 on a.id = a2.alert_id
    where is_invalid = 0
    and status='Fixed'
    and a2.actionability=0
    and a2.file_deleted is null
    and a2.file_renamed is null
    and a2.suppression is null
    and me.memory is null
    group by name
    order by name;
    '''
tnma=pd.DataFrame(sql.execute(q))
df = reduce(lambda x,y: pd.merge(x,y, on='name', how='outer'),[ta,tma,tnma])
df.memory= round((df.memory/df.total) * 100,2)
df.non_memory= round((df.non_memory/df.total) * 100,2)
df['memory']=df['memory'].astype(str)+"%"
df['non_memory']=df['non_memory'].astype(str)+"%"
df=df.rename(columns={'total':'elimination unknown'})
df

,name,elimination unknown,memory,non_memory
0,Firefox,3038,37.39%,62.61%
1,Kodi,302,14.24%,85.76%
2,Linux,4874,42.0%,58.0%
3,ovirt-engine,348,31.32%,68.68%
4,Samba,1928,29.67%,70.33%
